# Learning to rank on Microsoft Learning to Rank Datasets


In [ ]:
%matplotlib inline
from os.path import expanduser, join
from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score

from sklearn.externals import joblib
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression

## Loading and preparing the dataset

This is a NumPy array version of Fold1 of the [MSLR-WEB10K](http://research.microsoft.com/en-us/projects/mslr/) dataset.

In [2]:
%%time

import scipy.sparse
"""sparse_matrix = scipy.sparse.csc_matrix
"""

xrange = range

# Functions to extrac the documents, query and rank information
def extractFeatures(split):
    features = []
    for i in xrange(2, 138):
        features.append(float(split[i].split(':')[1]))
    # Convert to tuples:
    return features

def extractQueryData(split):
    # Add tuples:
    queryFeatures = split[1].split(':')[1]
    return queryFeatures

def readDataset(path):
    print('Reading training data from file...')
    with open(path, 'r') as file:
        #k=0
        features_list=[]
        rank_list=[]
        query_list=[]
        for line in file:
            split = line.split()
            features_list.append(extractFeatures(split))
            rank_list.append(int(split[0]))
            query_list.append(extractQueryData(split))
            #k+=1
            #if k==100:
            #    break
    print('Number of query ID %d' %(len(features_list)))
    return features_list,rank_list,query_list


X_train, y_train, qid_train = readDataset('datas/train.txt')

X_test, y_test, qid_test = readDataset('datas/test.txt')


"""X_train, y_train, qid_train = data['X_train'], data['y_train'], data['qid_train']
X_vali, y_vali, qid_vali = data['X_vali'], data['y_vali'], data['qid_vali']
X_test, y_test, qid_test = data['X_test'], data['y_test'], data['qid_test']"""

Reading training data from file...
Number of query ID 723412
Reading training data from file...
Number of query ID 241521
CPU times: user 1min 35s, sys: 2.16 s, total: 1min 37s
Wall time: 1min 37s


In [3]:

X_valid, y_valid, qid_valid = readDataset('datas/vali.txt')

Reading training data from file...
Number of query ID 235259


In [4]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
qid_train  = np.asarray(qid_train)

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)
qid_test  = np.asarray(qid_test)

X_valid = np.asarray(X_valid)
y_valid = np.asarray(y_valid)
qid_valid  = np.asarray(qid_valid)

Total size in bytes, total number of search results and number of queries:

In [5]:
len(X_train) + len(X_valid) + len(X_test)

1200192

In [6]:
len(np.unique(qid_train)) + len(np.unique(qid_valid)) + len(np.unique(qid_test))

10000

Concatenate the training and validation sets as a big development set.

In [7]:
X_dev = np.vstack([X_train, X_valid])
y_dev = np.concatenate([y_train, y_valid])
qid_dev = np.concatenate([qid_train, qid_valid])

In [8]:
print(X_dev.shape)
print(y_train.shape)


(958671, 136)
(723412,)


In [9]:
X_dev.dtype

dtype('float64')

Extract a subset of 500 queries to speed up the learning when prototyping

In [10]:
def subsample(X, y, qid, size, seed=None):
    rng = np.random.RandomState(seed)
    unique_qid = np.unique(qid)
    qid_mask = rng.permutation(len(unique_qid))[:size]
    subset_mask = np.in1d(qid_train, unique_qid[qid_mask])
    print(subset_mask)
    a =  y[subset_mask]
    return X[subset_mask,:],a, qid[subset_mask]


X_train_small, y_train_small, qid_train_small = subsample(
    X_train, y_train, qid_train, 500, seed=0)

[False False False ... False False False]


In [11]:
X_train_small.shape

(62655, 136)

Sanity check:

In [12]:
len(np.unique(qid_train_small))

500

In [13]:
X_train_medium, y_train_medium, qid_train_medium = subsample(
    X_train, y_train, qid_train, 1000, seed=0)

[False False False ... False False False]


In [14]:
def balance_irrelevant(X, y, qid, seed=None):
    """Subsample the zero-scored entries"""
    rng = np.random.RandomState(seed)
    unique_qid = np.unique(qid)
    final_mask = np.ones(shape=y.shape, dtype=np.bool)
    for this_qid in unique_qid:
        this_mask = qid == this_qid
        this_y = y[this_mask]
        relevant = this_y >= 2
        ratio = float(np.mean(relevant))
        if ratio > 0.5:
            # already balanced
            continue
            
        final_mask[this_mask] = np.logical_or(
            relevant, np.random.random(len(this_y)) > 0.7) 
    return X[final_mask], y[final_mask], qid[final_mask]

X_balanced_small, y_balanced_small, qid_balanced_small = balance_irrelevant(
    X_train_small, y_train_small, qid_train_small)

In [15]:
print(len(y_train_small))
print(len(y_balanced_small))

62655
26531


## Quantifying ranking success with NDCG

### NDCG explanation : https://en.wikipedia.org/wiki/Discounted_cumulative_gain

In [16]:
def dcg(relevances, rank=10):
    """Discounted cumulative gain at rank (DCG)"""
    relevances = np.asarray(relevances)[:rank]
    n_relevances = len(relevances)
    if n_relevances == 0:
        return 0.

    discounts = np.log2(np.arange(n_relevances) + 2)
    return np.sum(relevances / discounts)
 
def ndcg(relevances, rank=10):
    """Normalized discounted cumulative gain (NDGC)"""
    best_dcg = dcg(sorted(relevances, reverse=True), rank)
    if best_dcg == 0:
        return 0.

    return dcg(relevances, rank) / best_dcg

In [17]:
ndcg([2, 4, 0, 1, 1, 0, 0], rank=5)

0.8625300399291566

In [18]:
ndcg([0, 0, 0, 1, 1, 2, 4], rank=5)

0.13201850690866795

In [19]:
ndcg([0, 0, 0, 1, 1, 2, 4], rank=3)

0.0

In [20]:
ndcg([4, 2, 1, 1, 0, 0, 0], rank=5)

1.0

In [21]:
def mean_ndcg(y_true, y_pred, query_ids, rank=10):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    query_ids = np.asarray(query_ids)
    # assume query_ids are sorted
    ndcg_scores = []
    previous_qid = query_ids[0]
    previous_loc = 0
    for loc, qid in enumerate(query_ids):
        if previous_qid != qid:
            chunk = slice(previous_loc, loc)
            ranked_relevances = y_true[chunk][np.argsort(y_pred[chunk])[::-1]]
            ndcg_scores.append(ndcg(ranked_relevances, rank=rank))
            previous_loc = loc
        previous_qid = qid

    chunk = slice(previous_loc, loc + 1)
    ranked_relevances = y_true[chunk][np.argsort(y_pred[chunk])[::-1]]
    ndcg_scores.append(ndcg(ranked_relevances, rank=rank))
    return np.mean(ndcg_scores)

mean_ndcg([4, 3, 1, 4, 3], [4, 0, 1, 4, 2], [0, 0, 0, 2, 2], rank=10)

0.9795191506818377

In [22]:
def print_evaluation(model, X, y, qid):
    tic = time()
    y_predicted = model.predict(X)
    prediction_time = time() - tic
    print("Prediction time: {:.3f}s".format(prediction_time))
    print("NDCG@5 score: {:.3f}".format(
    mean_ndcg(y, y_predicted, qid, rank=5)))
    print("NDCG@10 score: {:.3f}".format(
    mean_ndcg(y, y_predicted, qid, rank=10)))
    print("NDCG score: {:.3f}".format(
    mean_ndcg(y, y_predicted, qid, rank=None)))
    print("R2 score: {:.3f}".format(r2_score(y, y_predicted)))

In [23]:
def plot_ndcg_by_trees(model, X, y, qid, rank=10):
    max_n_trees = len(model.estimators_)
    scores = []
    
    if hasattr(model, 'staged_predict'):
        # stage-wise score computation for boosted ensembles
        n_trees = np.arange(max_n_trees) + 1
        for y_predicted in model.staged_predict(X):
            scores.append(mean_ndcg(y, y_predicted, qid, rank=10))
    else:
        # assume forest-type of tree ensemble: use a log scale to speedup
        # the computation
        # XXX: partial predictions could be reused
        n_trees = np.logspace(0, np.log10(max_n_trees), 10).astype(int)
        for j, n in enumerate(n_trees):
            y_predicted = sub_ensemble(model, n).predict(X)
            scores.append(mean_ndcg(y, y_predicted, qid, rank=rank))
            
    plt.plot(n_trees, scores)
    plt.xlabel("Number of trees")
    plt.ylabel("Average NDCG@%d" % rank)
    _ = plt.title("Impact of the number of trees")

## Premier prédicteur : ExtraTreesRegressor

In [24]:
%%time

from sklearn.ensemble import ExtraTreesRegressor

etr = ExtraTreesRegressor(n_estimators=200, min_samples_split=5, random_state=1, n_jobs=-1)
etr.fit(X_train_small, y_train_small)

CPU times: user 7min 6s, sys: 1 s, total: 7min 7s
Wall time: 25.4 s


In [25]:
print_evaluation(etr, X_test, y_test, qid_test)

Prediction time: 1.608s
NDCG@5 score: 0.467
NDCG@10 score: 0.475
NDCG score: 0.729
R2 score: 0.138


## Learning to rank model : LambdaMart

See https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/MSR-TR-2010-82.pdf

In [ ]:
%%time

#!pip3 install pyltr

from pyltr.models import LambdaMART

#from sklearn.ensemble import LambdaMART

lmart= LambdaMART(n_estimators=300, max_depth=3,
                  learning_rate=0.1, random_state=1, verbose=1)
lmart.fit(X_train_small, y_train_small, qids=qid_train_small)

 Iter  Train score    Remaining                           Monitor Output 
    1       0.1784       30.46m                                         
    2       0.2388       30.60m                                         
    3       0.2533       30.96m                                         
    4       0.2605       30.92m                                         
    5       0.2656       30.96m                                         
    6       0.2665       30.90m                                         
    7       0.2695       30.72m                                         
    8       0.2702       30.46m                                         
    9       0.2706       30.51m                                         
   10       0.2704       30.36m                                         
   15       0.2748       29.77m                                         
   20       0.3040       29.43m                                         
   25       0.3060       29.01m                   

In [ ]:
print_evaluation(lmart, X_test, y_test, qid_test)

In [ ]:
print_evaluation(lmart, X_train_small, y_train_small, qid_train_small)

In [ ]:
%%time

lmart= LambdaMART(n_estimators=300, max_depth=3,
                  learning_rate=0.1, random_state=1, verbose=1)
lmart.fit(X_dev, y_dev, group=qid_dev)

In [ ]:
print_evaluation(lmart, X_test, y_test, qid_test)

In [ ]:
print_evaluation(lmart, X_dev, y_dev, qid_dev)

## Comparing with a baseline linear regression models (with different optimizers and input scaling)

In [ ]:
%time lr = LinearRegression().fit(X_dev, y_dev)

In [ ]:
%time y_test_lr = lr.predict(X_test)

In [ ]:
print_evaluation(lr, X_test, y_test, qid_test)

### RankSVM

En vous inspirant du code ici : https://gist.github.com/coreylynch/4150976/1a2983d3a896f4caba33e1a406a5c48962e606c0
- Programmez SVMRank
- Evaluez le sur les données de ce notebook
